In [1]:
pip install pyspark

In [2]:
pip install spark

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement spark (from versions: none)
ERROR: No matching distribution found for spark


In [ ]:
from datetime import date
import numpy as np
import pyspark.sql.functions as F

In [ ]:
def get_confusionmatrix(df_pred, label_col='label', prediction_col='predication'):
    df_pred.select(prediction_col,label_col)\
        .withColumn('TP', (F.col(prediction_col)==F.lit(1)) & (F.col(label_col)==F.lit(1))).cast('int')\
        .withColumn('TN', (F.col(prediction_col)==F.lit(0)) & (F.col(label_col)==F.lit(0))).cast('int')\
        .withColumn('FP', (F.col(prediction_col)==F.lit(1)) & (F.col(label_col)==F.lit(0))).cast('int')\
        .withColumn('FN', (F.col(prediction_col)==F.lit(0)) & (F.col(label_col)==F.lit(1))).cast('int')

Prepare data for model

In [ ]:
from spark.ml.feature import VectorAssembler

In [ ]:
df = spark.sql("""select * from customer_t""")

In [ ]:
# train test split
(df_train, df_test) = df.randomSplit([0.8, 0.2])

Scale data

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
# features:
inputCols = ['col1','col2','col3']
assembler = VectorAssembler(inputCols=inputCols, outputCol='features')
scaler = StandardScaler(inputCol='features', outputCol='label')

df_train = assembler.transform(df_train).select('label','features')
scaler_model = scaler.fit(df_train)
df_train = scaler_model.transform(df_train)

# target:
df_test = assembler.transform(df_test).select('label','features')
df_test = scaler_model.transform(df_test)

Run tree based model

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [ ]:
dtc_model = dtc.fit(df_train)
rfc_model = rfc.fit(df_train)
gbt_model = gbt.fit(df_train)

Evaluate

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [ ]:
dtc_preds = dtc_model.transform(df_test)
rfc_preds = rfc_model.transform(df_test)
gbt_preds = gbt_model.transform(df_test)

In [ ]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')
binary_eval = BinaryClassificationEvaluator()

In [ ]:
for model_name, model_pred in zip(['DecisionTreeClassifier','RandomForestClassifier','GBTClassifier'], [dtc_preds,rfc_preds,gbt_preds]):
    print(model_name)
    print('Accuracy: ', round(acc_eval.evaluate(model_pred),4))
    print('areaUnderROC: ', round(binary_eval.evaluate(model_pred),4))
    print('------'*5)

Feature importance

In [ ]:
for model_name, model in zip(['DecisionTreeClassifier','RandomForestClassifier','GBTClassifier'], [dtc_model,rfc_model,gbt_model]):
    print(model_name, " Feature Importance: ")
    for v, c in zip(inputCols, model.featureImportances):
        print(v, round(c, 6))

Confusion matrix

In [ ]:
for model_name, model_pred in zip(['DecisionTreeClassifier','RandomForestClassifier','GBTClassifier'], [dtc_preds,rfc_preds,gbt_preds]):
    print(model_name)
    get_confusionmatrix(model_pred)
    print("----"*5)